# POC for NCAABB Elo

In [1]:
cd ..

c:\Users\charl\Desktop\Git\elo-rating


c:\Users\charl\Desktop\Git\elo-rating\venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
from elo_runner import main
main()

Starting Runner for basketball/mens-college-basketball (2024-2024)...
Making Elo for basketball/mens-college-basketball - 2024
Starting Runner for football/college-football (2023-2023)...
Making Elo for football/college-football - 2023
Starting Runner for baseball/college-baseball (2024-2024)...
Making Elo for baseball/college-baseball - 2024
Starting Runner for hockey/mens-college-hockey (2024-2024)...
Making Elo for hockey/mens-college-hockey - 2024
Starting Runner for lacrosse/mens-college-lacrosse (2024-2024)...
Making Elo for lacrosse/mens-college-lacrosse - 2024
Starting Runner for basketball/nba (2024-2024)...
Making Elo for basketball/nba - 2024
Starting Runner for football/nfl (2023-2023)...
Making Elo for football/nfl - 2023
Starting Runner for baseball/mlb (2024-2024)...
Making Elo for baseball/mlb - 2024
Starting Runner for hockey/nhl (2024-2024)...
Making Elo for hockey/nhl - 2024
[Errno 2] No such file or directory: './data/elo/hockey/nhl/2005.parquet'
Starting Runner for

In [3]:
from src.consts import ESPNSportTypes, START_SEASONS, SEASON_GROUPS
from src.utils import find_year_for_season, create_dataframe, put_dataframe, get_dataframe
from events_runner import run_events_for_sport
from elo_runner import run_elo_for_sport
from src.event import ESPNEventsAPI
import numpy as np
import pandas as pd
import os
import datetime

## If its refresh pull whole thing
## If its upsert get the latest seasons in the fs and determine active sports that need pulling

sports = [
    ESPNSportTypes.COLLEGE_BASKETBALL,
    ESPNSportTypes.COLLEGE_FOOTBALL,
    ESPNSportTypes.COLLEGE_BASEBALL,
    ESPNSportTypes.COLLEGE_HOCKEY,
    ESPNSportTypes.COLLEGE_LACROSSE,
    ESPNSportTypes.NFL,
    ESPNSportTypes.NBA,
    ESPNSportTypes.NHL,
    ESPNSportTypes.MLB,
    ESPNSportTypes.PLL

]
import time
status_reports = {}
for sport in sports:
    start = time.time()
    try:
        run_events_for_sport(root_path='./data/events', sport=sport, espn_events_api=ESPNEventsAPI())
        run_elo_for_sport('./data/events','./data/elo',sport=sport)
        status_reports[sport] = {
            'status':True,
            'execution_time':round(time.time() - start, 2),
            'end_datetime': datetime.datetime.utcnow()
        }
    except Exception as e:
        print('FAILURE')
        print(e)
        status_reports[sport] = {
            'status':False,
            'execution_time':round(time.time() - start, 2),
            'end_datetime': datetime.datetime.utcnow()
        }
print('')
print('Events Pump Status Report')
print('-'*110)
duration = 0
for key, report in status_reports.items():
    duration = duration + report['execution_time']
    print(f"    {key}: {'PASSED' if report['status'] else 'FAILED'} -- took {report['execution_time']} sec, finished at ({report['end_datetime']}) ")
print('')
print(f'Pump took {duration} sec')
print('-'*110)



Starting Runner for basketball/mens-college-basketball (2024-2024)...
Getting Events for basketball/mens-college-basketball - 2024
    Updating data from 20240407-20240410...
Starting Runner for basketball/mens-college-basketball (2024-2024)...
Making Elo for basketball/mens-college-basketball - 2024
Starting Runner for football/college-football (2023-2023)...
Getting Events for football/college-football - 2023
    Updating data from 20240107-20240110...
Starting Runner for football/college-football (2023-2023)...
Making Elo for football/college-football - 2023
Starting Runner for baseball/college-baseball (2024-2024)...
Getting Events for baseball/college-baseball - 2024
    Updating data from 20240427-20240608...
Starting Runner for baseball/college-baseball (2024-2024)...
Making Elo for baseball/college-baseball - 2024
Starting Runner for hockey/mens-college-hockey (2024-2024)...
Getting Events for hockey/mens-college-hockey - 2024
    Updating data from 20240411-20240508...
Startin

In [ ]:
sports = [ESPNSportTypes.COLLEGE_BASKETBALL, ESPNSportTypes.COLLEGE_BASEBALL,ESPNSportTypes.COLLEGE_FOOTBALL,ESPNSportTypes.COLLEGE_HOCKEY]
SCHEMA = {
            'id':np.int64,
            'season':np.int32,
            'is_postseason': np.int8,
            'tournament_id':'Int32',
            'is_finished': np.int8,
            'neutral_site': np.int8,
            'home_team_id':np.int32,
            'home_team_score':'Int32',
            'away_team_id':np.int32,
            'away_team_score':'Int32',
        }
for sport in sports:
    for file in os.listdir(f'./data/events/{sport.value}'):
        fs_df = get_dataframe(f'./data/events/{sport.value}/{file}')
        put_dataframe(fs_df, f'./data/events/{sport.value}/{file}', SCHEMA)




In [3]:
from src.consts import ESPNSportTypes, START_SEASONS, SEASON_GROUPS
from src.event import ESPNEventsAPI
from src.sport import ESPNSport
sport = ESPNSportTypes.MLB
espn_sport_obj = ESPNSport(sport=sport, season=2024)
on_days = espn_sport_obj.ondays
espn_events_api=ESPNEventsAPI()
for i in on_days:
    res = espn_events_api.get_events_for_elo(sport,i)

In [42]:
season = 2024
import pandas as pd
import datetime
from src.consts import ESPNSportTypes, SEASON_GROUPS
from src.event import ESPNEventsAPI
from src.sport import ESPNSport
from src.utils import create_dataframe, put_dataframe, get_dataframe, get_seasons_to_update, known_missed_date
def get_valid_team_ids_for_sport_season(sport: ESPNSportTypes, season: int, espn_events_api: ESPNEventsAPI):
    """
    Get valid team IDs for a specific sport and season.

    Args:
        sport (ESPNSportTypes): Type of sport.
        season (int): Season year.
        espn_events_api (ESPNEventsAPI): ESPN Events API object.

    Returns:
        List: List of valid team IDs.
    """
    team_ids = []
    core_sport = sport.value.split('/')[0] + '/leagues/' + sport.value.split('/')[1]
    url = f'http://sports.core.api.espn.com/v2/sports/{core_sport}/seasons/{season}/teams'
    res = espn_events_api.api_request(url + '?limit=750')
    for item in res['items']:
        team_ids.append(int(item['$ref'].replace(url + '/', '').split('?')[0]))
    return team_ids


espn_events_api=ESPNEventsAPI()
sport = ESPNSportTypes.NFL
team_ids = get_valid_team_ids_for_sport_season(sport, season, espn_events_api)
espn_sport_obj = ESPNSport(sport=sport, season=season)
on_days = espn_sport_obj.ondays
groups = SEASON_GROUPS[sport]
if groups is not None:
    groups = groups['di']
if espn_sport_obj.start_date is None:
    print(f'No Data for {sport.value} - {season}...')
    
root_path = './data/events'
print(f'Getting Events for {sport.value} - {season}')
start_date = espn_sport_obj.start_date.strftime('%Y%m%d')
end_date = espn_sport_obj.end_date.strftime('%Y%m%d')

fs_df = get_dataframe(f'{root_path}/{sport.value}/{season}.parquet')
# If the file does not exist refresh else update the dataset with the new data
if fs_df.shape[0] == 0:
    fs_df = pd.DataFrame()
    dates = f"{start_date}-{end_date}"
    print(f'    Refreshing data from {dates}...')
else:
    # Upsert will be shifted back two days to make sure we do not miss any games
    try:
        start_date = pd.Timestamp(pd.Timestamp(fs_df.loc[fs_df['is_finished'] == True].datetime.max()).to_pydatetime() - datetime.timedelta(days=7)).strftime('%Y%m%d')
        
        
        dates = f"{start_date}-{end_date}"
        print(f'    Updating data from {dates}...')
        on_days = [day for day in on_days if pd.Timestamp(start_date).to_pydatetime() <= day <= pd.Timestamp(end_date).to_pydatetime()]

        fs_df = fs_df.loc[((fs_df.season == season) & (fs_df.is_finished == True) & (fs_df.datetime <= fs_df.loc[fs_df['is_finished'] == True].datetime.max()))]
    except Exception as e:
        print(f'Issue with Upsert for {sport.value} - {season}. Handling as refresh for Season...')
        fs_df = pd.DataFrame()
        dates = f"{start_date}-{end_date}"

events = []

Getting Events for football/nfl - 2024
    Updating data from 20240910-20250215...


In [36]:
missed_dates = []
for date in on_days:
    try:
        res = espn_events_api.get_events_for_elo(sport, date.strftime('%Y%m%d'), groups=groups)
        events.extend(res)
    except Exception as e:
        print(f"    -- Missed Date {date.strftime('%Y%m%d')} --")
        if date > datetime.datetime.utcnow():
            print(f'        Missed Date is future date. Scrape will be run up to {date - datetime.timedelta(days=1)}')
            break
        if not known_missed_date(sport, date):
            missed_dates.append(date)
# Add second pass (api timesout on a specific date sometimes. Finishing on_dates and re running missed dates results in less errors)
for date in missed_dates:
    res = espn_events_api.get_events_for_elo(sport, date.strftime('%Y%m%d'), groups=groups)
    events.extend(res)
df = create_dataframe(events, espn_events_api.SCHEMA)

df = df.loc[((df.away_team_id.isin(team_ids)) & (df.home_team_id.isin(team_ids)))].copy()

In [46]:
fs_df = get_dataframe(f'{root_path}/{sport.value}/{season}.parquet')
fs_df

,id,str_event_id,season,is_postseason,tournament_id,is_finished,neutral_site,date,datetime,home_team_id,home_team_name,home_team_score,away_team_id,away_team_name,away_team_score
0,401671789,20240906_chiefs_ravens,2024,0,<NA>,1,0,2024-09-06,2024-09-06 00:40:00+00:00,12,chiefs,27,33,ravens,20
1,401671805,20240907_eagles_packers,2024,0,<NA>,1,1,2024-09-07,2024-09-07 00:15:00+00:00,21,eagles,34,9,packers,29
2,401671744,20240908_falcons_steelers,2024,0,<NA>,1,0,2024-09-08,2024-09-08 17:00:00+00:00,1,falcons,10,23,steelers,18
3,401671617,20240908_bills_cardinals,2024,0,<NA>,1,0,2024-09-08,2024-09-08 17:00:00+00:00,2,bills,34,22,cardinals,28
4,401671719,20240908_bears_titans,2024,0,<NA>,1,0,2024-09-08,2024-09-08 17:00:00+00:00,3,bears,24,10,titans,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,401671838,20250105_broncos_chiefs,2024,0,<NA>,0,0,2025-01-05,2025-01-05 05:00:00+00:00,7,broncos,<NA>,12,chiefs,<NA>
273,401671840,20250105_cowboys_commanders,2024,0,<NA>,0,0,2025-01-05,2025-01-05 05:00:00+00:00,6,cowboys,<NA>,28,commanders,<NA>
272,401671827,20250105_falcons_panthers,2024,0,<NA>,0,0,2025-01-05,2025-01-05 05:00:00+00:00,1,falcons,<NA>,29,panthers,<NA>
278,401671837,20250105_colts_jaguars,2024,0,<NA>,0,0,2025-01-05,2025-01-05 05:00:00+00:00,11,colts,<NA>,30,jaguars,<NA>


In [45]:
a = pd.concat([fs_df, df], ignore_index=True).drop_duplicates(['str_event_id'], keep='last')
a

,id,str_event_id,season,is_postseason,tournament_id,is_finished,neutral_site,date,datetime,home_team_id,home_team_name,home_team_score,away_team_id,away_team_name,away_team_score
0,401671789,20240906_chiefs_ravens,2024,0,<NA>,1,0,2024-09-06,2024-09-06 00:40:00+00:00,12,chiefs,27,33,ravens,20
1,401671805,20240907_eagles_packers,2024,0,<NA>,1,1,2024-09-07,2024-09-07 00:15:00+00:00,21,eagles,34,9,packers,29
2,401671744,20240908_falcons_steelers,2024,0,<NA>,1,0,2024-09-08,2024-09-08 17:00:00+00:00,1,falcons,10,23,steelers,18
3,401671617,20240908_bills_cardinals,2024,0,<NA>,1,0,2024-09-08,2024-09-08 17:00:00+00:00,2,bills,34,22,cardinals,28
4,401671719,20240908_bears_titans,2024,0,<NA>,1,0,2024-09-08,2024-09-08 17:00:00+00:00,3,bears,24,10,titans,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,401671841,20250105_eagles_giants,2024,0,<NA>,0,0,2025-01-05,2025-01-05 05:00:00+00:00,21,eagles,<NA>,19,giants,<NA>
284,401671845,20250105_cardinals_49ers,2024,0,<NA>,0,0,2025-01-05,2025-01-05 05:00:00+00:00,22,cardinals,<NA>,25,49ers,<NA>
285,401671836,20250105_steelers_bengals,2024,0,<NA>,0,0,2025-01-05,2025-01-05 05:00:00+00:00,23,steelers,<NA>,4,bengals,<NA>
286,401671828,20250105_buccaneers_saints,2024,0,<NA>,0,0,2025-01-05,2025-01-05 05:00:00+00:00,27,buccaneers,<NA>,18,saints,<NA>
